In [18]:
from tensorflow.keras import Input, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, ReLU, MaxPooling2D

In [19]:
def residual_block(n_filters, x):
    """ Create a Residual Block of Convolutions
        n_filters: number of the filters.
        x : input into the block.
    """
    shortcut = x
    x = Conv2D(n_filters, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(x)
    x = Conv2D(n_filters, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu')(x)
    x = layers.add([shortcut, x])
    return x

In [20]:
def conv_block(n_filters, x):
    """ Create Block of Convolutions without Pooling
        n_filters: number of filters.
        x: input into block.
    """
    x = Conv2D(n_filters, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')(x)
    x = Conv2D(n_filters, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')(x)
    return x

In [21]:
# The input tensor
inputs = Input(shape=(224,224,3)) 
# First convolutional layer, where pooled feature maps will be reduced by 75%
x = Conv2D(64, kernel_size=(7,7), strides=(2,2), padding='same', activation='relu')(inputs)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

# First residual block group of 64 filters
for _ in range(2):
    x = residual_block(64, x)
# Doubles the size of filters and reduces feature maps by 75% (strides=(2,2)) to fit the next residual block
x = conv_block(128, x)

for _ in range(3):
    x = residual_block(128, x)
x = conv_block(256, x)

for _ in range(5):
    x = residual_block(256, x)
x = conv_block(512, x)

x = residual_block(512, x)

x = layers.GlobalAveragePooling2D()(x)
outputs = Dense(1000, activation='softmax')(x)
model = Model(inputs, outputs)

In [22]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_58 (Conv2D)             (None, 112, 112, 64  9472        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 56, 56, 64)  0           ['conv2d_58[0][0]']              
                                                                                            

                                                                                                  
 conv2d_82 (Conv2D)             (None, 4, 4, 256)    590080      ['conv2d_81[0][0]']              
                                                                                                  
 add_31 (Add)                   (None, 4, 4, 256)    0           ['add_30[0][0]',                 
                                                                  'conv2d_82[0][0]']              
                                                                                                  
 conv2d_83 (Conv2D)             (None, 2, 2, 512)    1180160     ['add_31[0][0]']                 
                                                                                                  
 conv2d_84 (Conv2D)             (None, 1, 1, 512)    2359808     ['conv2d_83[0][0]']              
                                                                                                  
 conv2d_85